**Import needed packages/modules**

In [ ]:
# Cell 1
import matplotlib.pyplot as plt
import numpy as np
from numba import float64, int64, vectorize

**Declare a numba accelerated function that computes the Halton QRNG**
1. The parameter $n$ is an integer of any size
2. The parameter $p$ is a prime number

In [ ]:
# Cell 2
@vectorize([float64(int64, int64)], nopython=True)
def halton(n, p):
    h, f = 0, 1
    while n > 0:
        f = f / p
        h += (n % p) * f
        n = int(n / p)
    return h

**Set the number of random $dots$ (samples) to take**

In [ ]:
# Cell 3
total_dots =

**Take $n$ "random" samples of 2D Cartesian points $(x,y)$ using the Halton sequence**
1. The Halton QRNG returns a random float [0,1)
2. Subtract that float from 1, so the interval flips to (0,1] ensuring any points on the curve will now contribute to the area
3. Scale the result so $-2\le x\le2$ and $0\le y\le 5$

In [ ]:
# Cell 4
x = (1 - halton(np.arange(total_dots), 2)) *
y = (1 - halton(np.arange(total_dots), 3)) *
print(x)
print(y)

**Create an array $d$ that contains the distance above/below the curve $y=-x^2+4$ for every point $(x,y)$**
1. $d=$ (the curve's $y$ value at $x$) - (the sampled $y$ value at $x$)
2. Leverage the fact the exponentiation and addition operators are "vector aware"

In [ ]:
# Cell 5
d =
print(d)

**Create arrays of $(x,y)$ coordinates that are "on or below" vs. "above" the parabola using the Pythagorean distance $d$**
1. If the "random" $y$ value at $x>$ the curve height value at $x$, then $d<0$
2. If the "random" $y$ value at $x\le$ the curve height value at $x$, then $d\ge 0$
2. Leverage the ability to `filter` numpy arrays using a conditional expression

In [ ]:
# Cell 6

x_in = x[d >= 0.0] # On or below the curve
y_in = y[d >= 0.0]

x_out = x[d < 0.0] # Above the curve
y_out = y[d < 0.0]

**Calculate the absolute percent error in the area estimation**
1. The actual/expected area of a unit circle is exactly $\large\frac{32}{2}$
2. The sample area is $(-2\le x\le 2)\times(0\le y\le 5)=20$
3. The "inside" dots are where $d\ge 0$
4. Therefore the observed/estimated area using the Monte Carlo formulation $\large=20 \times\frac{dots_{\ inside}}{dots_{\ total}}$


In [ ]:
# Cell 7
act =
est =
err = np.abs((est - act) / act)

print(f"dots = {total_dots:,}")
print(f"act = {act:.6f}")
print(f"est = {est:.6f}")
print(f"err = {err:.5%}")

**Display the scatter plot of the Monte Carlo estimation**

In [ ]:
# Cell 8
plt.scatter(x_in, y_in, color="red", marker=".", s=0.5)
plt.scatter(x_out, y_out, color="blue", marker=".", s=0.5)
plt.title("$y=-x^2+4$")
plt.xlabel("x")
plt.ylabel("y")
plt.show()